# Usage of Dask vectors and Dask operators on a Kubernetes cluster

@Author: Ettore Biondi - ebiondi@caltech.edu

This notebook shows how to start a Dask client using an existing Kubernetes cluster and run any method present within any Dask-based class. The interface we use is based on the Dask Kubernetes one (https://kubernetes.dask.org/en/latest/). This process's power is given by the fact that the same syntax as the local cluster is used to run similar operations. However, all the computations are performed within the remote Kubernetes cluster. 

### Importing necessary libraries

In [1]:
import numpy as np
import occamypy
#Plotting library
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
# %matplotlib inline
params = {
    'image.interpolation': 'nearest',
    'image.cmap': 'gray',
    'savefig.dpi': 300,  # to adjust notebook inline plot size
    'axes.labelsize': 14, # fontsize for x and y labels (was 10)
    'axes.titlesize': 14,
    'font.size': 14,
    'legend.fontsize': 14,
    'xtick.labelsize': 14,
    'ytick.labelsize': 14,
    'text.usetex':True
}
matplotlib.rcParams.update(params)

WARNING! DATAPATH not found. The folder /tmp will be used to write binary files


Before starting, make sure you have a running Kubernetes cluster. In our case, we are using a Kubernetes cluster created employing the Google Cloud Platform (GCP). Any other Cloud provider can be used to use the Dask Kubernetes interface since it based on Kubernetes native commands. First, let's check how many nodes in the cluster are available by running the following command:

In [2]:
!kubectl get nodes

NAME                                    STATUS   ROLES    AGE   VERSION
gke-test-cluster-pool-1-c7981ec8-3mst   Ready    <none>   98m   v1.16.13-gke.401
gke-test-cluster-pool-1-c7981ec8-s42f   Ready    <none>   98m   v1.16.13-gke.401
gke-test-cluster-pool-1-c7981ec8-tnks   Ready    <none>   98m   v1.16.13-gke.401
gke-test-cluster-pool-1-c7981ec8-xxdd   Ready    <none>   98m   v1.16.13-gke.401


where we see that we have 4 nodes. 

Let's now instantiate the Dask client object using the occamypy interface. For more information run "help(occamypy.DaskClient)". Currently, we employ kubefwd (https://github.com/txn2/kubefwd) to allow communication between the local machine and the Kubernetes cluster. Therefore, one needs to run in the background the following command:

```sudo kubefwd svc -n default -n kube-system ```

which requires root access. We are currently working on finding a solution to this limitation. Once, this command is running in the background, it is possible to start the Dask-Kubernetes cluster and client. The following command will take a few moment to start since a container for the scheduler and each worker needs to be created.
The user can check this process by running the following kubectl command:

```kubectl get pods```

In [3]:
client_params = {"memory_limit":'1G', "memory_request":'1G',"cpu_limit":1, "cpu_request":1}
client = occamypy.DaskClient(kube_params=client_params, n_wrks=3)

Creating scheduler pod on cluster. This may take some time.


Similarly as in the [Dask-based classes](./Dask-based&#32;classes.ipynb) tutorial, let's check the number of workers and run some simple computation using the Dask-Kubernetes cluster.

In [4]:
print("Number of workers = %d"%client.getNworkers())
print("Workers Ids = %s"%client.getWorkerIds())

Number of workers = 3
Workers Ids = ['tcp://10.68.0.8:44173', 'tcp://10.68.1.8:33953', 'tcp://10.68.2.10:45987']


### Dask vectors
The same exact syntax can be used to perform vector operations as in the other examples. However, all the computations are performed on the Kubernetes cluster.

In [5]:
vec_temp = occamypy.VectorNumpy((200, 340))
chunks = (5, 3, 6) # 5 vectors to worker 1; 3 vectors to worker 2; ...
vecD = occamypy.DaskVector(client, vector_template=vec_temp, chunks=chunks)

Again, let's check some of the vector chunks' properties and vector operations.

In [6]:
# shape
print("List of shapes: %s" % vecD.shape)
# Randomize
vecD.rand()
# Norm
print("Dask vector norm = %s" % vecD.norm())
# Scaling
vecD.scale(10)
print("Scaled Dask vector norm = %s" % vecD.norm())
# Cloning
vecD1 = vecD.clone()
# Summing two vectors
vecD1 + vecD
# Check norm
print("Sum Dask vector norm = %s" % vecD1.norm())

List of shapes: [(340, 200), (340, 200), (340, 200), (340, 200), (340, 200), (340, 200), (340, 200), (340, 200), (340, 200), (340, 200), (340, 200), (340, 200), (340, 200), (340, 200)]
Dask vector norm = 563.6544056088085
Scaled Dask vector norm = 5636.544056088079
Sum Dask vector norm = 11273.088112176158


### Dask operators
Let's now try the Dask operators using the Kubernetes cluster. We are going to test the same operations as in the other Dask interface tutorial.

In [7]:
# Construct a simple scaling operator acting on each chunk of a Dask Vector
vec = occamypy.VectorNumpy((100, 25))
chunks = (20, 5, 12)
sc = 10.0
vecD = occamypy.DaskVector(client, vector_template=vec, chunks=chunks)
# Creating list of lists of the arguments for the operator's constructor
scal_op_args = [(vec_i, sc) for vec_i in vecD.vecDask]

# Instantiating Dask operator
scaleOpD = occamypy.DaskOperator(client, occamypy.Scaling, scal_op_args, chunks)

In [8]:
# Dot-product test
scaleOpD.dotTest(True)
# Power method
max_eig = scaleOpD.powerMethod()
print("\nMaximum eigenvalue = %s" % max_eig)

Dot-product tests of forward and adjoint operators
--------------------------------------------------
Applying forward operator add=False
 Runs in: 0.4625699520111084 seconds
Applying adjoint operator add=False
 Runs in: 0.44726991653442383 seconds
Dot products add=False: domain=2.758842e+03 range=2.758842e+03 
Absolute error: 2.273737e-12
Relative error: 8.241636e-16 

Applying forward operator add=True
 Runs in: 0.47759199142456055 seconds
Applying adjoint operator add=True
 Runs in: 0.4708428382873535 seconds
Dot products add=True: domain=5.517683e+03 range=5.517683e+03 
Absolute error: 2.728484e-12
Relative error: 4.944981e-16 

-------------------------------------------------

Maximum eigenvalue = 9.999999999999998


In [9]:
vecD.rand()
vecD1 = scaleOpD.getRange().clone()
scaleOpD.forward(False, vecD, vecD1)
print("Norm of the input = %s" % vecD.norm())
print("Norm of the output = %s" % vecD1.norm())

Norm of the input = 175.27081379444783
Norm of the output = 1752.7081379444776


In [10]:
S = occamypy.DaskSpread(client, vec, chunks)
S.dotTest(True) # checking dot-product

Dot-product tests of forward and adjoint operators
--------------------------------------------------
Applying forward operator add=False
 Runs in: 4.349317789077759 seconds
Applying adjoint operator add=False
 Runs in: 0.6235499382019043 seconds
Dot products add=False: domain=8.813910e+01 range=8.813910e+01 
Absolute error: 4.263256e-14
Relative error: 4.836964e-16 

Applying forward operator add=True
 Runs in: 3.8977253437042236 seconds
Applying adjoint operator add=True
 Runs in: 0.43741893768310547 seconds
Dot products add=True: domain=1.762782e+02 range=1.762782e+02 
Absolute error: 2.842171e-14
Relative error: 1.612321e-16 

-------------------------------------------------


In [11]:
#Chain of scaling and spreading operator
scale_S = scaleOpD * S
scale_S.dotTest(True) # checking dot-product
# Testing product of Dask Operators
x = vec.rand()
y = scale_S.getRange().clone()
scale_S.forward(False, x, y)
print("\nFirst element of x = %s" % x.getNdArray()[0,0])
print("First element of y = %s" % y.getNdArray()[0][0,0])

Dot-product tests of forward and adjoint operators
--------------------------------------------------
Applying forward operator add=False
 Runs in: 4.366874933242798 seconds
Applying adjoint operator add=False
 Runs in: 1.1945748329162598 seconds
Dot products add=False: domain=1.974351e+03 range=1.974351e+03 
Absolute error: 1.818989e-12
Relative error: 9.213098e-16 

Applying forward operator add=True
 Runs in: 4.2433929443359375 seconds
Applying adjoint operator add=True
 Runs in: 1.0472321510314941 seconds
Dot products add=True: domain=3.948703e+03 range=3.948703e+03 
Absolute error: 2.728484e-12
Relative error: 6.909824e-16 

-------------------------------------------------

First element of x = 0.6018041355008692
First element of y = 6.018041355008692
